In [1]:
import pandas as pd
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from nltk.util import ngrams
from nltk.chunk import RegexpParser
import contractions
import re
import emoji
from typing import List, Tuple
from sklearn.model_selection import train_test_split
import os

In [2]:
# Cleaning functions
def clean_text(text: str) -> str:
    """Clean raw text by removing URLs, emojis, mentions, and MBTI codes."""
    text = text.lower()
    text = re.sub(r'https?\S+|www\S+', '', text)
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'@\w+|#', '', text)
    text = re.sub(r"[^a-z\']", ' ', text)
    # Remove MBTI type codes (e.g., INFJ, ENTP) to avoid leaking information
    text = re.sub(r'\b(I|E)(N|S)(F|T)(J|P)\b', '', text, flags=re.IGNORECASE)
    # Remove common footer
    text = re.sub(r'\bsent (from )?my \w+(\s\w+)? using tapatalk\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'w w w', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def preprocess_posts(posts_str: str) -> str:
    """Split multi-post string (separated by '|||'), clean each, and join."""
    posts = posts_str.split('|||')
    cleaned_posts = [clean_text(post) for post in posts]
    joined = ' '.join(cleaned_posts)
    return re.sub(r'\s+', ' ', joined).strip()

In [3]:
# Encoding function
def encode_mbti_type(mbti: str) -> Tuple[int, int, int, int]:
    """
    Encode MBTI type as 4 binary dimensions:
    I/E → 1/0, N/S → 1/0, F/T → 1/0, J/P → 1/0
    """
    return (
        1 if mbti[0] == 'I' else 0,
        1 if mbti[1] == 'N' else 0,
        1 if mbti[2] == 'F' else 0,
        1 if mbti[3] == 'J' else 0,
    )

In [4]:
# Initialize NLTK components
STOP_WORDS = set(stopwords.words('english'))
WNL = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

def get_wordnet_pos(tag: str) -> str:
    """Map POS tag to WordNet format for lemmatization."""
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mayasalkhateeb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/mayasalkhateeb/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mayasalkhateeb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mayasalkhateeb/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mayasalkhateeb/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
# Tokenization variants
def tokens_without_lemma(text: str) -> List[str]:
    fixed = contractions.fix(text)
    tokens = word_tokenize(fixed)
    filtered_tokens = [word for word in tokens if word.lower() not in STOP_WORDS]
    return filtered_tokens

def tokens_with_lemma(text: str) -> List[str]:
    fixed = contractions.fix(text)
    tokens = word_tokenize(fixed)
    filtered_tokens = [word for word in tokens if word.lower() not in STOP_WORDS]
    lemmatized = [WNL.lemmatize(word.lower()) for word in filtered_tokens]
    return lemmatized

def tokens_with_lemma_pos(text: str) -> List[str]:
    fixed = contractions.fix(text)
    tokens = word_tokenize(fixed)
    filtered_tokens = [word for word in tokens if word.lower() not in STOP_WORDS]
    pos_tags = pos_tag(filtered_tokens)
    lemmatized = [
        WNL.lemmatize(token.lower(), pos=get_wordnet_pos(pos))
        for token, pos in pos_tags
    ]
    return lemmatized

def tokens_with_dependency_tree(text: str) -> List[str]:
    """
    Extract simple dependency-like triples using heuristic rules (e.g., subj-verb-obj).
    Returns list of strings like 'subj-verb-obj'.
    """
    fixed = contractions.fix(text)
    sentences = sent_tokenize(fixed)
    relations = []
    for sent in sentences:
        tokens = word_tokenize(sent)
        if not tokens:
            continue
        pos = pos_tag(tokens)
        for i in range(len(pos)):
            if pos[i][1].startswith('VB'):
                subj = None
                obj = None
                # Look left for subject (NN or PR)
                for j in range(i - 1, -1, -1):
                    if pos[j][1].startswith('NN') or pos[j][1].startswith('PR'):
                        subj = pos[j][0]
                        break
                # Look right for object (NN or PR)
                for j in range(i + 1, len(pos)):
                    if pos[j][1].startswith('NN') or pos[j][1].startswith('PR'):
                        obj = pos[j][0]
                        break
                if subj and obj:
                    relation = f"{subj}-{pos[i][0]}-{obj}"
                    relations.append(relation)
    return relations

def tokens_with_chunking(text: str) -> List[str]:
    """
    Extract chunks (NP, PP, VP) using RegexpParser.
    Returns list of chunk phrases as strings.
    """
    fixed = contractions.fix(text)
    sentences = sent_tokenize(fixed)
    chunks = []
    grammar = r"""
        NP: {<DT|JJ|NN.*>+}          # Noun phrases
        PP: {<IN><NP>}               # Prepositional phrases
        VP: {<VB.*><NP|PP>*}         # Verb phrases
    """
    cp = RegexpParser(grammar)
    for sent in sentences:
        tokens = word_tokenize(sent)
        if not tokens:
            continue
        pos = pos_tag(tokens)
        tree = cp.parse(pos)
        for subtree in tree.subtrees(lambda t: t.label() in ['NP', 'PP', 'VP']):
            chunk_text = ' '.join(word for word, tag in subtree.leaves())
            chunks.append(chunk_text)
    return chunks

In [6]:
# N-gram generation
def generate_ngrams(tokens: List[str]) -> Tuple[List[Tuple], List[Tuple], List[Tuple]]:
    if not tokens:
        return [], [], []
    unigrams = list(ngrams(tokens, 1))
    bigrams = list(ngrams(tokens, 2))
    trigrams = list(ngrams(tokens, 3))
    return unigrams, bigrams, trigrams

In [7]:
# Main processing
df = pd.read_csv("../data/raw/mbti_1.csv")  # Adjust path if needed

print("🧹 Cleaning posts...")
tqdm.pandas()
df['cleaned_posts'] = df['posts'].progress_apply(preprocess_posts)

print("🏷️ Encoding MBTI types...")
df[['IE', 'NS', 'FT', 'JP']] = pd.DataFrame(
    df['type'].progress_apply(encode_mbti_type).tolist(),
    index=df.index
)

variants = [
    'without_lemma',
    'with_lemma',
    'with_lemma_pos',
    'with_dep_tree',
    'with_chunking'
]

for var in variants:
    print(f"🔤 Processing variant: {var}")
    if var == 'without_lemma':
        func = tokens_without_lemma
    elif var == 'with_lemma':
        func = tokens_with_lemma
    elif var == 'with_lemma_pos':
        func = tokens_with_lemma_pos
    elif var == 'with_dep_tree':
        func = tokens_with_dependency_tree
    elif var == 'with_chunking':
        func = tokens_with_chunking
    
    df[f'tokens_{var}'] = df['cleaned_posts'].progress_apply(func)
    
    print(f"📊 Generating n-grams for {var}...")
    ngram_results = df[f'tokens_{var}'].progress_apply(generate_ngrams)
    df[[f'Unigrams_{var}', f'Bigrams_{var}', f'Trigrams_{var}']] = pd.DataFrame(
        ngram_results.tolist(), index=df.index
    )

# Drop intermediate columns
df = df.drop(columns=['posts', 'cleaned_posts'])



🧹 Cleaning posts...


100%|██████████| 8675/8675 [00:57<00:00, 150.83it/s]


🏷️ Encoding MBTI types...


100%|██████████| 8675/8675 [00:00<00:00, 1114583.77it/s]


🔤 Processing variant: without_lemma


...

100%|██████████| 8675/8675 [00:04<00:00, 1819.13it/s]


🔤 Processing variant: with_chunking


100%|██████████| 8675/8675 [07:21<00:00, 19.66it/s]


📊 Generating n-grams for with_chunking...


100%|██████████| 8675/8675 [00:02<00:00, 3941.47it/s]


In [9]:
# Split into train and test
print("✂️ Splitting data into train/test...")
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['type'])

# Save to ../data/processed
os.makedirs('../data/processed', exist_ok=True)
train_df.to_pickle('../data/processed/train.pkl')
test_df.to_pickle('../data/processed/test.pkl')

print("✅ Processing complete! Data saved to ../data/processed/train.pkl and test.pkl")

✂️ Splitting data into train/test...
✅ Processing complete! Data saved to ../data/processed/train.pkl and test.pkl
